Develop an algorithm to predict fraud. Prioritize correctly finding fraud rather than correctly labeling non-fraudulent transactions.

Dataset: https://www.kaggle.com/dalpozz/creditcardfraud
(code modified from # modified from: https://www.kaggle.com/joparga3/in-depth-skewed-data-classif-93-recall-acc-now)

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.font_manager


In [2]:
df = pd.read_csv('creditcard.csv')
df.head()
# no null values
#print(df.isnull().sum())

# V features are the output of PCA (cannot provide original dataset do to confidentiality)
print(df.columns)

# many more nonfradulent transactions than fradulent
#print(df.groupby('Class')['Time'].nunique()) 

print(df.groupby('Class')['Class'].count())


Index(['Time', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10',
       'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20',
       'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'Amount',
       'Class'],
      dtype='object')
Class
0    284315
1       492
Name: Class, dtype: int64


In [3]:
df.dtypes
df.head(n=150)

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0
5,2.0,-0.425966,0.960523,1.141109,-0.168252,0.420987,-0.029728,0.476201,0.260314,-0.568671,...,-0.208254,-0.559825,-0.026398,-0.371427,-0.232794,0.105915,0.253844,0.081080,3.67,0
6,4.0,1.229658,0.141004,0.045371,1.202613,0.191881,0.272708,-0.005159,0.081213,0.464960,...,-0.167716,-0.270710,-0.154104,-0.780055,0.750137,-0.257237,0.034507,0.005168,4.99,0
7,7.0,-0.644269,1.417964,1.074380,-0.492199,0.948934,0.428118,1.120631,-3.807864,0.615375,...,1.943465,-1.015455,0.057504,-0.649709,-0.415267,-0.051634,-1.206921,-1.085339,40.80,0
8,7.0,-0.894286,0.286157,-0.113192,-0.271526,2.669599,3.721818,0.370145,0.851084,-0.392048,...,-0.073425,-0.268092,-0.204233,1.011592,0.373205,-0.384157,0.011747,0.142404,93.20,0
9,9.0,-0.338262,1.119593,1.044367,-0.222187,0.499361,-0.246761,0.651583,0.069539,-0.736727,...,-0.246914,-0.633753,-0.120794,-0.385050,-0.069733,0.094199,0.246219,0.083076,3.68,0


In [4]:
print(df.isnull().sum())

Time      0
V1        0
V2        0
V3        0
V4        0
V5        0
V6        0
V7        0
V8        0
V9        0
V10       0
V11       0
V12       0
V13       0
V14       0
V15       0
V16       0
V17       0
V18       0
V19       0
V20       0
V21       0
V22       0
V23       0
V24       0
V25       0
V26       0
V27       0
V28       0
Amount    0
Class     0
dtype: int64


In [5]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(C=1e9)

Y = df.Class
X = df.drop(['Class'],axis=1)
lr.fit(X,Y)

LogisticRegression(C=1000000000.0, class_weight=None, dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

In [40]:
pred_y_sklearn = lr.predict(X)
table = pd.crosstab(Y,pred_y_sklearn)
print('Prediction Confusion Matrix:')

print(table)

print('\n Percentage accuracy')
print(lr.score(X, Y))

# same thing:
#print('\n Percentage accuracy')
#print((table.iloc[0,0] + table.iloc[1,1]) / (table.sum().sum()))

print('\n Recall:')
# Recall = TP/(TP+FN)
print((table.iloc[1,1]) / (table.iloc[1,1] + table.iloc[1,0]))

Prediction Confusion Matrix:
col_0       0    1
Class             
0      284240   75
1         203  289

 Percentage accuracy
0.999023900396

 Recall:
0.587398373984


### Initial result conclusions:
The recall (the percentage of time the model correctly identifies fraudulent accounts) is low at 59%. This is likely because of class imbalance. I'll try training on a sample with a reduced majority class

In [28]:
# modified from: https://www.kaggle.com/joparga3/in-depth-skewed-data-classif-93-recall-acc-now
# Number of data points in the minority class
number_records_fraud = len(df[df.Class == 1])
fraud_indices = np.array(df[df.Class == 1].index)

# Picking the indices of the normal classes
normal_indices = df[df.Class == 0].index

# Out of the indices we picked, randomly select "x" number (number_records_fraud) - getting the same number of non-fradulent
# to fraudulent
random_normal_indices = np.random.choice(normal_indices, number_records_fraud, replace = False)
random_normal_indices = np.array(random_normal_indices)

# Appending the 2 indices
under_sample_indices = np.concatenate([fraud_indices,random_normal_indices])

# Under sample dataset
under_sample_data = df.iloc[under_sample_indices,:]

X_undersample = under_sample_data.ix[:, under_sample_data.columns != 'Class']
Y_undersample = under_sample_data.ix[:, under_sample_data.columns == 'Class']
Y_US = Y_undersample['Class'] 

In [29]:
# splitting the dataset between training/testing

from sklearn.cross_validation import train_test_split

# Whole dataset
X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size = 0.3, random_state = 0)

print("Number transactions train dataset: ", len(X_train))
print("Number transactions test dataset: ", len(X_test))
print("Total number of transactions: ", len(X_train)+len(X_test))

# Undersampled dataset
X_train_undersample, X_test_undersample, y_train_undersample, y_test_undersample = train_test_split(X_undersample\
                                                                                                   ,Y_undersample\
                                                                                                   ,test_size = 0.3\
                                                                                                   ,random_state = 0)
print("")
print("Number transactions train dataset: ", len(X_train_undersample))
print("Number transactions test dataset: ", len(X_test_undersample))
print("Total number of transactions: ", len(X_train_undersample)+len(X_test_undersample))

Number transactions train dataset:  199364
Number transactions test dataset:  85443
Total number of transactions:  284807

Number transactions train dataset:  688
Number transactions test dataset:  296
Total number of transactions:  984


In [31]:
lr_undersample = LogisticRegression(C=1e9)
lr_undersample.fit(X_train_undersample,y_train_undersample['Class'])

LogisticRegression(C=1000000000.0, class_weight=None, dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

In [39]:
pred_y_undersample = lr_undersample.predict(X_test_undersample)
table_undersample = pd.crosstab(y_test_undersample['Class'],pred_y_undersample)
print('Prediction Confusion Matrix:')

print(table_undersample)

print('\n Percentage accuracy')
print(lr_undersample.score(X_test_undersample, y_test_undersample['Class']))

# same thing:
#print('\n Percentage accuracy')
#print((table.iloc[0,0] + table.iloc[1,1]) / (table.sum().sum()))

print('\n Recall:')
# Recall = TP/(TP+FN)
print((table_undersample.iloc[1,1]) / (table_undersample.iloc[1,1] + table_undersample.iloc[1,0]))

Prediction Confusion Matrix:
col_0    0    1
Class          
0      145    4
1       16  131

 Percentage accuracy
0.932432432432

 Recall:
0.891156462585


### Conclusions from test of undersampled dataset
testing the model on the undersampled dataset improved the recall to 89% - much better. Next I want to see how it performs on the whole dataset

In [41]:
# try new model with the full dataset
pred_y_fulldataset = lr_undersample.predict(X_test)
table_fulldataset = pd.crosstab(y_test,pred_y_fulldataset)
print('Prediction Confusion Matrix:')

print(table_fulldataset)

print('\n Percentage accuracy')
print(lr_undersample.score(X_test, y_test))

print('\n Recall:')
# Recall = TP/(TP+FN)
print((table_fulldataset.iloc[1,1]) / (table_fulldataset.iloc[1,1] + table_fulldataset.iloc[1,0]))

Prediction Confusion Matrix:
col_0      0     1
Class             
0      82827  2469
1         20   127

 Percentage accuracy
0.970869468535

 Recall:
0.863945578231


### Conclusion
Ways to make up for class imbalance:
1. Resample the dataset with more balanced classes for training
2. Create synthetic records to generate more of the minority classes for training


In this case I used some of the code from https://www.kaggle.com/joparga3/in-depth-skewed-data-classif-93-recall-acc-now to improve the recall on my model